In [1]:
!pip install selenium


# Big Data Projekt_Gruppe 4

### Lebensmittelpreisanalyse: Billa, Hofer & Interspar

In diesem Projektabschnitt wurden **Produktdaten aus österreichischen Supermärkten (Billa, Hofer und Interspar)** gesammelt. Fokus lag auf den Kategorien:

- Brot & Gebäck  
- Kühlwaren  
- Fleisch & Fisch  

Mittels **Web Scraping** wurden die Preise und Produktnamen aus den jeweiligen Onlineshops extrahiert und als strukturierte **CSV-Dateien** gespeichert. Diese Dateien dienen als Grundlage für die spätere Analyse, Visualisierung und Vergleichbarkeit der Preise zwischen den Supermärkten.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time

# Kategorie-URLs
urls = [
    "https://www.interspar.at/shop/lebensmittel/wurst-fleisch-fisch/c/F3/",
    "https://www.interspar.at/shop/lebensmittel/kuehlregal/c/F2/",
    "https://www.interspar.at/shop/lebensmittel/brot-gebaeck/c/F6/"
]

driver = webdriver.Chrome()
base_url = "https://www.interspar.at"
product_list = []

try:
    for url in urls:
        driver.get(url)
        time.sleep(2)

        while True:
            # Warten auf Produkte
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "productBox"))
            )
            time.sleep(1)

            # HTML einlesen
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            product_boxes = soup.find_all('div', class_='productBox')

            for box in product_boxes:
                link_url = box.get('data-url')
                link = base_url + link_url if link_url else ""

                # Preis zusammensetzen aus zwei <label>
                try:
                    price_container = box.find('div', class_='actualPriceContainer')
                    euros = price_container.find('label', class_='priceInteger').text.strip()
                    cents = price_container.find('label', class_='priceDecimal').text.strip()
                    price = f"{euros},{cents} €"
                except:
                    price = "Kein Preis"

                # Produktname (zweiter Titel ist Name)
                title_tags = box.find_all('div', class_='productTitle')
                if len(title_tags) > 1 and title_tags[1].has_attr('title'):
                    product_name = title_tags[1]['title'].strip()
                elif title_tags and title_tags[0].has_attr('title'):
                    product_name = title_tags[0]['title'].strip()
                else:
                    product_name = "Kein Name"

                product_list.append({
                    'Kategorie': url.split("/")[-3],  # z. B. kuehlregal
                    'Produkt': product_name,
                    'Preis': price,
                    'Link': link
                })

            # Weiterblättern, falls möglich
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="nächste Seite"]')
                if "disabled" in next_button.get_attribute("class"):
                    break
                next_button.click()
                time.sleep(2)
            except:
                break

finally:
    driver.quit()

# Speichern als CSV
with open('interspar_kategorien.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Kategorie', 'Produkt', 'Preis', 'Link'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte erfolgreich gespeichert in 'interspar_kategorien.csv'")






In [100]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Liste der Billa-Kategorien
urls = [
    "https://shop.billa.at/kategorie/kuehlwaren-13841",
    "https://shop.billa.at/kategorie/brot-und-gebaeck-13766"
]

driver = webdriver.Chrome()
product_list = []

for url in urls:
    driver.get(url)

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "ws-product-item-base"))
    )
    time.sleep(3)

    products = driver.find_elements(By.CLASS_NAME, "ws-product-item-base")
    print(f"{len(products)} Produkte gefunden auf {url}")

    for product in products:
        # Produktname aus Attribut
        try:
            product_name = product.get_attribute("data-teaser-name")
        except:
            product_name = "Kein Name"

        # Preis extrahieren
        try:
            price = product.find_element(By.CLASS_NAME, "ws-product-price-type__value").text.strip()
        except:
            price = "Kein Preis"

        product_list.append({'Produkt': product_name, 'Preis': price, 'Seite': url})

driver.quit()

# CSV speichern
with open('billa_mehrere_seiten.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Produkt', 'Preis', 'Seite'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte gespeichert.")



30 Produkte gefunden auf https://shop.billa.at/kategorie/kuehlwaren-13841
30 Produkte gefunden auf https://shop.billa.at/kategorie/brot-und-gebaeck-13766
60 Produkte gespeichert.


In [82]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Starte Browser
driver = webdriver.Chrome()
driver.get("https://www.hofer.at/de/sortiment/produktsortiment/fleisch-und-fisch.html")

# Warte bis Produktkarten sichtbar sind
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "item.plp_product"))
)
time.sleep(2)

# Finde alle Produkte
products = driver.find_elements(By.CLASS_NAME, "item.plp_product")
print(f"Gefundene Produkte: {len(products)}")

product_list = []

for product in products:
    # Produktname aus h2
    try:
        name_tag = product.find_element(By.CLASS_NAME, "product-title")
        product_name = name_tag.text.strip()
    except:
        product_name = "Kein Name"

    # Preis aus span
    try:
        price_tag = product.find_element(By.CLASS_NAME, "at-product-price_lbl")
        price = price_tag.text.strip()
    except:
        price = "Kein Preis"

    product_list.append({'Produkt': product_name, 'Preis': price})

driver.quit()

# Export als CSV
with open('hofer_produkte.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Produkt', 'Preis'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte gespeichert in 'hofer_produkte.csv'")


Gefundene Produkte: 100
100 Produkte gespeichert in 'hofer_produkte.csv'


In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Kategorien-URLs
urls = [
    "https://www.hofer.at/de/sortiment/produktsortiment/fleisch-und-fisch.html",
    "https://www.hofer.at/de/sortiment/produktsortiment/kuehlung.html",
    "https://www.hofer.at/de/sortiment/produktsortiment/vorratsschrank.html"
]

# Starte Browser
driver = webdriver.Chrome()
product_list = []

try:
    for url in urls:
        driver.get(url)
        time.sleep(2)

        while True:
            try:
                # Warte auf Produkte
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "item.plp_product"))
                )
                time.sleep(1)

                products = driver.find_elements(By.CLASS_NAME, "item.plp_product")
                print(f"{len(products)} Produkte auf Seite in Kategorie: {url.split('/')[-1]}")

                for product in products:
                    # Produktname
                    try:
                        name_tag = product.find_element(By.CLASS_NAME, "product-title")
                        product_name = name_tag.text.strip()
                    except:
                        product_name = "Kein Name"

                    # Preis
                    try:
                        price_tag = product.find_element(By.CLASS_NAME, "at-product-price_lbl")
                        price = price_tag.text.strip()
                    except:
                        price = "Kein Preis"

                    # Zur Liste hinzufügen
                    product_list.append({
                        'Kategorie': url.split('/')[-1].replace('.html', ''),
                        'Produkt': product_name,
                        'Preis': price
                    })

                # Weiter-Button prüfen
                next_button = driver.find_element(By.CLASS_NAME, "pagination-next")
                if "disabled" in next_button.get_attribute("class"):
                    break
                else:
                    next_button.click()
                    time.sleep(2)

            except:
                break

finally:
    driver.quit()

# Speichere alles als CSV
with open('hofer_kategorien.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Kategorie', 'Produkt', 'Preis'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte aus allen Kategorien gespeichert in 'hofer_kategorien.csv'")



100 Produkte auf Seite in Kategorie: fleisch-und-fisch.html
100 Produkte auf Seite in Kategorie: kuehlung.html
100 Produkte auf Seite in Kategorie: vorratsschrank.html
300 Produkte aus allen Kategorien gespeichert in 'hofer_kategorien.csv'


In [2]:
!pip install pandas pymongo

In [8]:
import pandas as pd
from pymongo import MongoClient

# Verbindung zur MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["supermarkt_preise"]
collection = db["lebensmittel"]

# Optional: Bestehende Daten vorher löschen
collection.delete_many({})

# Deine Dateien + zugehöriger Supermarktname
files_with_market = {
    "C:/Users/pinar/DataspellProjects/BDProject/Projekt/lebensmittelpreise-bigdata/data/billa_mehrere_seiten.csv": "Billa",
    "C:/Users/pinar/DataspellProjects/BDProject/Projekt/lebensmittelpreise-bigdata/data/hofer_kategorien.csv": "Hofer",
    "C:/Users/pinar/DataspellProjects/BDProject/Projekt/lebensmittelpreise-bigdata/data/interspar_kategorien.csv": "Interspar"
}

# Alle in dieselbe Collection importieren
for filepath, supermarkt in files_with_market.items():
    try:
        df = pd.read_csv(filepath)
        df["Supermarkt"] = supermarkt  
        data = df.to_dict(orient="records")
        collection.insert_many(data)
        print(f"{len(data)} Datensätze aus '{supermarkt}' importiert.")
    except Exception as e:
        print(f"Fehler bei Datei {filepath}: {e}")


60 Datensätze aus 'Billa' importiert.
300 Datensätze aus 'Hofer' importiert.
240 Datensätze aus 'Interspar' importiert.
